In [8]:
pip install pysam 


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pysam
import pandas as pd

In [ ]:

vcf_file_path = '/content/drive/My Drive/Colab Notebooks/raw_variants.vcf.gz'#or insert local filepath here

# Open the file using pysam or vcfpy as shown earlier
vcf_file = pysam.VariantFile(vcf_file_path, "r")
 # bam = pysam.VariantFile(Processed_WGS, "rb") for a .bam file

In [ ]:
#FOR INDEXING PURPOSES ONLY
#pysam.tabix_index(vcf_file_path, preset="vcf")
#print("Indexing complete. Retrying APOE4 identifier.")
#vcf = pysam.VariantFile(vcf_file_path, "r")  # Reopen VCF file after indexing

In [ ]:
def APOE4_identifier(vcf):
    """
        Checks for the APOE4 mutation seperately from the other mutations due to how highly correlated to AD it is.
        Parameters:
        vcf_file (pysam.VariantFile): The VCF file containing genomic variants.

        Returns:
        PRS(float): The PRS contribution from APOE4 based on the risk allele count and beta coefficient.
    """
    apoe4_position = 44908684  # APOE4 SNP position on chromosome 19
    apoe4_risk_allele = "C"    # Risk allele associated with Alzheimer's
    apoe4_beta = 1.20177
    apoe4_OR = 3.326
    print("Checking for APOE4 allele...")
    chr19 = "chr19"
    PRS_dueto_Apoe4 = 0

    try:
        for record in vcf_file.fetch(chr19, apoe4_position-1, apoe4_position):
            # Check if mutation is found at the position (you can add additional checks here)
            print(f"Mutation found at {chr19}:{apoe4_position}: {record}")
            genotype = record.samples[0]['GT']  # Extract genotype (0,1) or (1,1))
            alleles = [record.alleles[g] for g in genotype if g is not None]
            apoe4_allele_count = sum(1 for allele in alleles if allele == apoe4_risk_allele)

                # Calculate PRS contribution
            prs_contribution = apoe4_beta * apoe4_allele_count
            PRS_dueto_Apoe4 += prs_contribution

            print(f"Genotype at APOE4 ({chr19}:{apoe4_position}): {alleles}")
            print(f"Risk Allele Count: {apoe4_allele_count}, PRS Contribution: {PRS_dueto_Apoe4}")
            return  PRS_dueto_Apoe4

    except ValueError as e:
        print(f"Error fetching data for {chr19}:{apoe4_position} - {str(e)}")
        return None  # Skip this contig if an error occurs

        #add print outside to say no mutations in chr 19"

    #vcf.close()



In [ ]:
gwas_summary = pd.read_excel('/content/drive/My Drive/Colab Notebooks/cleaned_gwas.xlsx')

PRS = 0

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
def chrom_loc(gwas_summary, vcf, PRS):
    """
        Finds SNP locations in a genomic sequence, and checks wether they have the risk allele or not.
        Parameters:
        vcf_file (pysam.VariantFile): The VCF file containing genomic variants.
        gwas_summary(Pandas Database): Contains the mutations, the chromosomal coordinates
        PRS: Default Polygenic Risk Score (0)

        Returns:
        PRS(float): The PRS contribution from SNPs based on the risk allele count and beta coefficient.

    """
    for index, row in gwas_summary.iterrows():  # Iterate through GWAS summary
        chrom, pos = row['locations'].split(":")  # Split 'chromosome:position'
        beta = float(row['beta'])  # Extract beta value
        risk_allele = row['risk Allele']

        # Add 'chr' prefix if not already present
        if not chrom.startswith("chr"):
            chrom = "chr" + chrom

        # Check if the contig exists in the VCF header
        if chrom not in vcf_file.header.contigs:
            print(f"Skipping invalid contig: {chrom}")
            continue  # Skip this contig if it's not valid

        try:
            for record in vcf_file.fetch(chrom, int(pos)-1, int(pos)):
                genotype = record.samples[0]['GT']  # Extract genotype tuple

                # Calculate genotype risk allele count (sum of GT alleles)
                risk_allele_count = sum(gt for gt in genotype if gt is not None)

                # Calculate PRS contribution
                prs_contribution = beta * risk_allele_count
                PRS += prs_contribution

                print(f"Mutation at {chrom}:{pos}, Genotype: {genotype}, Risk Allele: {risk_allele}, PRS Contribution: {prs_contribution}")

        except ValueError as e:
            print(f"Error fetching data for {chrom}:{pos} - {str(e)}")
            continue  # Skip this contig if an error occurs


    print(f"Your Polygenic Risk Score is: {PRS}")
    return PRS

    #vcf.close()




In [ ]:
def chrom1_loc(gwas_summary, vcf):
    """
        Added another function to check wether mutated allele = risk allele as that was missing. now PRS is less though TT
        Finds SNP locations in a genomic sequence, and checks wether they have the risk allele or not.
        Parameters:
        vcf_file (pysam.VariantFile): The VCF file containing genomic variants.
        gwas_summary(Pandas Database): Contains the mutations, the chromosomal coordinates
        PRS: Default Polygenic Risk Score (0)

        Returns:
        PRS(float): The PRS contribution from SNPs based on the risk allele count and beta coefficient.

    """
    PRSSS = 0  # Initialize PRS score

    for index, row in gwas_summary.iterrows():  # Iterate through GWAS summary
        chrom, pos = row['locations'].split(":")  # Split 'chromosome:position'
        beta = float(row['beta'])  # Extract beta value
        risk_allele = row['risk Allele']  # Extract risk allele from GWAS

    # Add 'chr' prefix if not already present
        if not chrom.startswith("chr"):
            chrom = "chr" + chrom

    # Check if the contig exists in the VCF header
        if chrom not in vcf_file.header.contigs:
            print(f"Skipping invalid contig: {chrom}")
            continue  # Skip this contig if it's not valid

        try:
            for record in vcf_file.fetch(chrom, int(pos)-1, int(pos)):  # Fetch variants at position
                genotype = record.samples[0]['GT']  # Extract genotype (tuple of allele indices)
                alleles = [record.alleles[g] for g in genotype if g is not None]  # Convert to allele bases

            # Count only matching risk alleles
                risk_allele_count = sum(1 for allele in alleles if risk_allele in alleles)

            # Calculate PRS contribution
                prs_contribution = beta * risk_allele_count
                PRSSS += prs_contribution

                print(f"Mutation found at {chrom}:{pos}: Genotype={alleles}, Risk Allele={risk_allele} PRS Contribution={prs_contribution}")

        except ValueError as e:
            print(f"Error fetching data for {chrom}:{pos} - {str(e)}")
            continue  # Skip this contig if an error occurs

    print(f"Your Polygenic Risk Score is: {PRSSS}")
    return PRSSS







In [ ]:
def PRS_calc(PRS):
    """
        Calculate PRS and then uses a certain threshold to determine wether teh individual is high risk.
        Used Clumping and Threshold calulcation method, Otherwise known as PRS(C+T) method.
        Used previous research to determine the threshold of 0.00009735 PRS.
        Parameters:
        PRS: Polygenic Risk Score (Ammended by chromloc function)

        Returns:
        String: Genetic liability based on Normal Distribution documented in previous studies.


    """
    if PRS < -0.268:
      print("Your Polygenic Risk Score is in the first quartile! Your genetic liability for Alzheimer's is: low")
    if PRS > 0.1725:
      print("Your Polygenic Risk Score is in the fourth quartile! Your genetic liability for Alzheimer's is: elevated")
    if 0.1725 > PRS > -0.268:
      print("Your Polygenic Risk Score is average! Your genetic liability for Alzheimer's is: average")



In [ ]:
APOE4_identifier(vcf_file)

chrom_loc(gwas_summary, vcf_file, PRS)

chrom1_loc(gwas_summary, vcf_file)
PRS_calc(PRS)